# Examen 1

## Importaciones necesarias

In [1]:
import pandas as pd
import matplotlib as plt
import scipy as sp
import numpy as np

JSON de nombre "countries.json" descargado del siguiente link:
https://gist.githubusercontent.com/keeguon/2310008/raw/bdc2ce1c1e3f28f9cab5b4393c7549f38361be4e/countries.

In [4]:
NombreArchivo = "countries.json"
DatosCountries = pd.read_json(NombreArchivo)

NameError: name 'countries' is not defined